LangChain is a framework that helps us build applications powered by LLMs.
Instead of manually writing code for prompts, memory, retrieval, and tools, LangChain gives us modular components to combine everything into a working system.

LLMs alone can only generate text.
But real applications need:
• prompts
• external data
• memory
• tools
• retrieval (RAG)
• chain-of-thought workflows

LangChain helps us orchestrate all these pieces into an end-to-end pipeline.

Whenever we build a RAG or knowledge-based GenAI app, we always follow this pipeline

Source Document(s) ->

Chunking (splitting text into small pieces)->

Embeddings (convert chunks into vectors)->

Vectorstore (store vectors for semantic search)->

Retriever (find relevant chunks for a user query)->

LLM + retrieved context → Final Answer





In [ ]:
!pip install -U langchain-groq langchain-community langchain faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# LLM CALL WITH GROQ

In [ ]:
from langchain_groq import ChatGroq

# Create LLM instance
llm = ChatGroq(
    groq_api_key="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",
    model_name="llama-3.1-8b-instant"
)

# Basic call
resp = llm.invoke("Explain what LangChain is in two sentences.")
print(resp.content)


LangChain is an open-source framework that enables developers to build and train large language models by leveraging existing ones, allowing for more efficient and scalable model development. It uses a concept called "chain" to connect multiple models together, allowing for tasks like question answering, text generation, and conversational dialogue to be performed in a more modular and composable way.


# DOCUMENT LOADING (SOURCE)
LangChain provides Document Loaders to read PDFs, text files, websites, YouTube transcripts, etc.

In [ ]:
!pip install -U langchain langchain-community


In [ ]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Apple released the first iPhone in 2007."),
    Document(page_content="Samsung produces flagship smartphones with high refresh rate displays."),
    Document(page_content="Apples are nutritious fruits commonly eaten fresh.")
]

# TEXT SPLITTING (CHUNKING)


When documents are large, we must split them into smaller pieces (chunks).
This is needed because:

LLM context windows are limited.

Embeddings work best on small chunks.

Retrieval is more accurate when chunks are clean.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=30
)

chunks = splitter.split_documents(docs)
print("Number of chunks:", len(chunks))
chunks[:2]

Number of chunks: 3


[Document(metadata={}, page_content='Apple released the first iPhone in 2007.'),
 Document(metadata={}, page_content='Samsung produces flagship smartphones with high refresh rate displays.')]

# EMBEDDINGS


Embeddings convert text chunks into numerical vectors.
These vectors capture meaning and enable semantic search.”

We will use Groq embeddings soon, but Groq currently generates text models.
So for embeddings we use a common community embedding model like all-MiniLM through HuggingFace.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Example: embedding first chunk
vec = embed_model.embed_query(chunks[0].page_content)
len(vec), vec[:5]

(384,
 [-0.006448030937463045,
  -0.031008413061499596,
  0.08662212640047073,
  0.08914606273174286,
  -0.050914257764816284])

# VECTORSTORE (FAISS)


Vectorstores allow semantic search.
FAISS is fast, simple, and works entirely in memory.

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embed_model)


RETRIEVER


A retriever is a wrapper that performs similarity search on a vectorstore.
Given a user query, it finds the most relevant chunks

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

retrieved_docs = retriever.invoke("smartphone release")
retrieved_docs

[Document(id='37e67486-235b-4350-b8e6-be5b96377d7a', metadata={}, page_content='Apple released the first iPhone in 2007.'),
 Document(id='18ccd7b7-b0fb-4367-b7e8-73179ba61a5b', metadata={}, page_content='Samsung produces flagship smartphones with high refresh rate displays.')]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Answer the question using ONLY the context below.
If the answer is not in the context, say "I don't know."

Context:
{context}

Question:
{question}

Answer:
""")


In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)


In [ ]:
response = rag_chain.invoke("When was the first iPhone released?")
print(response.content)


In [ ]:
response = rag_chain.invoke("Who is the CEO of Apple?")
print(response.content)


Document Loaders in LangChain (Beyond Plain Text)

So far in this tutorial, we used simple text documents to demonstrate how a RAG pipeline works.
However, real-world GenAI applications rarely rely on plain text alone.

LangChain provides Document Loaders that allow us to ingest many different data sources and formats into a RAG system.

📄 Common Document Types Supported by LangChain

LangChain supports far more than text files, including:

📄 Text & Files

.txt files

.md (Markdown)

.csv

.json

Log files

📘 PDFs & Office Documents

PDF files

Word documents (.docx)

PowerPoint slides (.pptx)

Excel files (.xlsx)

🌐 Web & Online Content

Websites (HTML pages)

Blogs & documentation

Wikis

REST APIs

In [ ]:
from langchain_community.document_loaders import TextLoader


In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
from langchain_community.document_loaders import WebBaseLoader


In [ ]:
from langchain_community.document_loaders import YoutubeLoader
